In [1]:
%%javascript
IPython.notebook.kernel.execute('nb_name = "' + IPython.notebook.notebook_name + '"')

<IPython.core.display.Javascript object>

In [2]:
%load_ext autoreload
%autoreload
from IPython.display import clear_output

import os
os.environ['PYTHONHASHSEED'] = '0'
os.environ["NUMEXPR_NUM_THREADS"] = "1"
os.environ["MKL_NUM_THREADS"] = "1"
os.environ["OMP_NUM_THREADS"] = "1"

import sys
import argparse
import subprocess
import pdb
import time
import random
import _pickle as cPickle
import matplotlib.pyplot as plt
import glob

%matplotlib inline

import numpy as np
import pandas as pd
import tensorflow as tf

from data_structure import get_batches
from hntm import HierarchicalNeuralTopicModel
from tree import get_descendant_idxs
from evaluation import validate, get_hierarchical_affinity, get_topic_specialization, print_topic_sample
from configure import get_config

# load data & set config

In [3]:
config = get_config(nb_name)

In [4]:
os.environ["CUDA_VISIBLE_DEVICES"] = config.gpu
np.random.seed(config.seed)
random.seed(config.seed)

In [5]:
instances_train_tmp, instances_dev, instances_test, word_to_idx, idx_to_word, bow_idxs = cPickle.load(open(config.path_data,'rb'))

In [6]:
if len(instances_train_tmp) > config.size:
    instances_train = np.random.choice(instances_train_tmp, config.size, replace=False)
else:
    instances_train = instances_train_tmp

In [7]:
train_batches = get_batches(instances_train, config.batch_size)
dev_batches = get_batches(instances_dev, config.batch_size)
test_batches = get_batches(instances_test, config.batch_size)
config.dim_bow = len(bow_idxs)

In [8]:
def debug_shape(variables, model):
    sample_batch = test_batches[0][1]
    feed_dict = model.get_feed_dict(sample_batch)
    _variables = sess.run(variables, feed_dict=feed_dict)
    for _variable, variable in zip(_variables, variables):
        if hasattr(variable, 'name'):
            print(variable.name, ':', _variable.shape)
        else:
            print(_variable.shape)

def debug_value(variables, model, return_value=False):
    sample_batch = test_batches[0][1]
    feed_dict = model.get_feed_dict(sample_batch)
    _variables = sess.run(variables, feed_dict=feed_dict)

    return _variables

# run

## initialize log

In [9]:
checkpoint = []
losses_train = []
ppls_train = []
ppl_min = np.inf
epoch = 0
train_batches = get_batches(instances_train, config.batch_size, iterator=True)

log_df = pd.DataFrame(columns=pd.MultiIndex.from_tuples(
                    list(zip(*[['','','','TRAIN:','','','','','VALID:','','','','','TEST:','', 'SPEC:', '', '', 'HIER:', ''],
                            ['Time','Ep','Ct','LOSS','PPL','NLL','KL','REG','LOSS','PPL','NLL','KL','REG','LOSS','PPL', '1', '2', '3', 'CHILD', 'OTHER']]))))

cmd_rm = 'rm -r %s' % config.dir_model
res = subprocess.call(cmd_rm.split())
cmd_mk = 'mkdir %s' % config.dir_model
res = subprocess.call(cmd_mk.split())

def update_checkpoint(config, checkpoint, global_step):
    checkpoint.append(config.path_model + '-%i' % global_step)
    if len(checkpoint) > config.max_to_keep:
        path_model = checkpoint.pop(0) + '.*'
        for p in glob.glob(path_model):
            os.remove(p)
    cPickle.dump(checkpoint, open(config.path_checkpoint, 'wb'))

## initialize model

In [10]:
if 'sess' in globals(): sess.close()
model = HierarchicalNeuralTopicModel(config)
sess = tf.Session(config=tf.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1))
sess.run(tf.global_variables_initializer())
saver = tf.train.Saver(max_to_keep=config.max_to_keep)
update_tree_flg = False

## train & validate model

In [ ]:
time_start = time.time()
while epoch < config.n_epochs:
    # train
    time_start = time.time()
    for ct, batch in train_batches:
        feed_dict = model.get_feed_dict(batch)
        _, loss_batch, topic_loss_recon_batch, topic_loss_kl_batch, topic_loss_reg_batch, ppls_batch, global_step_log = \
        sess.run([model.opt, model.loss, model.topic_loss_recon, model.topic_loss_kl, model.topic_loss_reg, model.topic_ppls, tf.train.get_global_step()], feed_dict = feed_dict)

        losses_train += [[loss_batch, topic_loss_recon_batch, topic_loss_kl_batch, topic_loss_reg_batch]]
        ppls_train += list(ppls_batch)

    if epoch % 10 == 0:
        time_log = float(time.time() - time_start)
        
        # validate
        loss_train, topic_loss_recon_train, topic_loss_kl_train, topic_loss_reg_train = np.mean(losses_train, 0)
        ppl_train = np.exp(np.mean(ppls_train))
        loss_dev, topic_loss_recon_dev, topic_loss_kl_dev, topic_loss_reg_dev, ppl_dev, probs_topic_dev = validate(sess, dev_batches, model)

        # test
        if ppl_dev < ppl_min:
            ppl_min = ppl_dev
            loss_test, _, _, _, ppl_test, _ = validate(sess, test_batches, model)
            saver.save(sess, config.path_model, global_step=global_step_log)
            cPickle.dump(config, open(config.path_config % global_step_log, 'wb'))
            update_checkpoint(config, checkpoint, global_step_log)

        # visualize topic
        topics_freq_indices = np.argsort(sess.run(model.topic_bow), 1)[:, ::-1][:, :config.n_freq]
        topics_freq_idxs = bow_idxs[topics_freq_indices]
        topic_freq_tokens = {topic_idx: [idx_to_word[idx] for idx in topic_freq_idxs] for topic_idx, topic_freq_idxs in zip(model.topic_idxs, topics_freq_idxs)}
        topic_prob_topic = {topic_idx: prob_topic for topic_idx, prob_topic in zip(model.topic_idxs, probs_topic_dev)}
        descendant_idxs = {parent_idx: get_descendant_idxs(model, parent_idx) for parent_idx in model.topic_idxs}
        recur_prob_topic = {parent_idx: np.sum([topic_prob_topic[child_idx] for child_idx in recur_child_idxs]) for parent_idx, recur_child_idxs in descendant_idxs.items()}

        depth_specs = get_topic_specialization(sess, model, instances_test)
        hierarchical_affinities = get_hierarchical_affinity(sess, model)

        # log
        clear_output()
        log_series = pd.Series([time_log, epoch, ct, \
                '%.2f'%loss_train, '%.0f'%ppl_train, '%.2f'%topic_loss_recon_train, '%.2f'%topic_loss_kl_train, '%.2f'%topic_loss_reg_train, \
                '%.2f'%loss_dev, ppl_dev, '%.2f'%topic_loss_recon_dev, '%.2f'%topic_loss_kl_dev, '%.2f'%topic_loss_reg_dev, \
                '%.2f'%loss_test, ppl_test, \
                '%.2f'%depth_specs[1], '%.2f'%depth_specs[2], '%.2f'%depth_specs[3], \
                '%.2f'%hierarchical_affinities[0], '%.2f'%hierarchical_affinities[1]],
                index=log_df.columns)
        log_df.loc[global_step_log] = log_series
        display(log_df)
        cPickle.dump(log_df, open(os.path.join(config.path_log), 'wb'))
        print_topic_sample(sess, model, topic_prob_topic=topic_prob_topic, recur_prob_topic=recur_prob_topic, topic_freq_tokens=topic_freq_tokens)

        # update tree
        if not config.static:
            config.tree_idxs, update_tree_flg = model.update_tree(topic_prob_topic, recur_prob_topic)
            if update_tree_flg:
                print(config.tree_idxs)
                name_variables = {tensor.name: variable for tensor, variable in zip(tf.global_variables(), sess.run(tf.global_variables()))} # store paremeters
                if 'sess' in globals(): sess.close()
                model = HierarchicalNeuralTopicModel(config)
                sess = tf.Session(config=tf.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1))
                name_tensors = {tensor.name: tensor for tensor in tf.global_variables()}
                sess.run([name_tensors[name].assign(variable) for name, variable in name_variables.items()]) # restore parameters
                saver = tf.train.Saver(max_to_keep=1)

    train_batches = get_batches(instances_train, config.batch_size, iterator=True)
    epoch += 1

loss_dev, topic_loss_recon_dev, topic_loss_kl_dev, topic_loss_reg_dev, ppl_dev, probs_topic_dev = validate(sess, dev_batches, model)
topics_freq_indices = np.argsort(sess.run(model.topic_bow), 1)[:, ::-1][:, :config.n_freq]
topics_freq_idxs = bow_idxs[topics_freq_indices]
topic_freq_tokens = {topic_idx: [idx_to_word[idx] for idx in topic_freq_idxs] for topic_idx, topic_freq_idxs in zip(model.topic_idxs, topics_freq_idxs)}
topic_prob_topic = {topic_idx: prob_topic for topic_idx, prob_topic in zip(model.topic_idxs, probs_topic_dev)}
descendant_idxs = {parent_idx: get_descendant_idxs(model, parent_idx) for parent_idx in model.topic_idxs}
recur_prob_topic = {parent_idx: np.sum([topic_prob_topic[child_idx] for child_idx in recur_child_idxs]) for parent_idx, recur_child_idxs in descendant_idxs.items()}
display(log_df)
print_topic_sample(sess, model, topic_prob_topic=topic_prob_topic, recur_prob_topic=recur_prob_topic, topic_freq_tokens=topic_freq_tokens)

TRAIN:                            VALID:  \
            Time   Ep   Ct    LOSS   PPL     NLL    KL   REG    LOSS   
250     5.612293    0  249  113.84   639  113.34  0.47  0.03  105.44   
2750    2.912895   10  249  111.90   719  111.07  0.82  0.00  104.36   
5250    2.343413   20  249  111.33   971  110.21  1.11  0.00  104.00   
7750    2.530269   30  249  111.03  1148  109.75  1.28  0.00  103.83   
10250   2.464288   40  249  110.85  1256  109.45  1.39  0.00  103.77   
12750   2.779380   50  249  110.72  1322  109.24  1.47  0.01  103.71   
15250   2.618883   60  249  110.62  1376  109.07  1.54  0.01  103.68   
17750   2.805142   70  249  110.52  1418  108.92  1.59  0.01  103.59   
20250   2.449063   80  249  110.46  1443  108.82  1.63  0.01  103.60   
22750   2.788883   90  249  110.39  1467  108.71  1.67  0.01  103.46   
25250   2.828255  100  249  110.33  1489  108.61  1.71  0.01  103.40   
27750   2.834542  110  249  110.27  1509  108.51  1.75  0.00  103.34   
30250   2.754933  120  249  110.22  1524  108.42  1.79  0.00  103.29   
32750   2.869401  130  249  110.17  1536  108.34  1.83  0.00  103.25   
35250   2.968248  140  249  110.13  1544  108.26  1.86  0.00  103.22   
37750   2.962683  150  249  110.09  1549  108.20  1.89  0.00  103.19   
40250   2.758934  160  249  110.05  1552  108.13  1.91  0.00  103.18   
42750   3.676067  170  249  110.02  1553  108.08  1.94  0.00  103.20   
45250   3.096191  180  249  109.99  1553  108.03  1.96  0.00  103.19   
47750   2.975990  190  249  109.96  1553  107.98  1.98  0.00  103.18   
50250   2.631310  200  249  109.94  1553  107.93  2.00  0.00  103.17   
52750   2.913388  210  249  109.91  1552  107.89  2.02  0.00  103.16   
55250   2.763817  220  249  109.89  1551  107.85  2.04  0.00  103.16   
57750   2.936258  230  249  109.87  1550  107.81  2.05  0.00  103.15   
60250   3.076973  240  249  109.85  1549  107.78  2.07  0.00  103.14   
62750   2.942892  250  249  109.83  1547  107.74  2.08  0.00  103.13   
65250   2.666451  260  249  109.81  1546  107.71  2.09  0.00  103.14   
67750   2.945064  270  249  109.79  1545  107.68  2.11  0.00  103.12   
70250   2.721444  280  249  109.78  1544  107.65  2.12  0.00  103.13   
72750   2.864359  290  249  109.76  1542  107.63  2.13  0.00  103.11   
...          ...  ...  ...     ...   ...     ...   ...   ...     ...   
102750  2.777372  410  249  109.62  1523  107.38  2.24  0.00  103.08   
105250  2.591555  420  249  109.61  1522  107.37  2.24  0.00  103.09   
107750  2.777227  430  249  109.60  1520  107.35  2.25  0.00  103.08   
110250  2.616923  440  249  109.60  1518  107.34  2.25  0.00  103.09   
112750  3.766159  450  249  109.59  1517  107.32  2.26  0.00  103.08   
115250  2.588329  460  249  109.58  1515  107.31  2.27  0.00  103.09   
117750  3.393378  470  249  109.57  1513  107.30  2.27  0.00  103.07   
120250  2.864587  480  249  109.57  1512  107.28  2.28  0.00  103.09   
122750  2.807423  490  249  109.56  1510  107.27  2.28  0.00  103.07   
125250  2.685430  500  249  109.55  1509  107.26  2.29  0.00  103.08   
127750  3.229206  510  249  109.55  1507  107.25  2.29  0.00  103.07   
130250  3.526363  520  249  109.54  1506  107.24  2.30  0.00  103.08   
132750  2.741117  530  249  109.53  1504  107.23  2.30  0.00  103.06   
135250  3.762432  540  249  109.53  1503  107.22  2.31  0.00  103.08   
137750  2.788171  550  249  109.52  1502  107.20  2.31  0.00  103.06   
140250  2.695285  560  249  109.52  1500  107.19  2.31  0.00  103.08   
142750  2.789104  570  249  109.51  1499  107.18  2.32  0.00  103.06   
145250  2.575730  580  249  109.51  1498  107.18  2.32  0.00  103.08   
147750  2.838492  590  249  109.50  1496  107.17  2.33  0.00  103.06   
150250  2.669738  600  249  109.50  1495  107.16  2.33  0.00  103.08   
152750  2.803215  610  249  109.49  1494  107.15  2.33  0.00  103.06   
155250  3.034725  620  249  109.49  1493  107.14  2.34  0.00  103.07   
157750  2.776047  630  249  109.48  1492  107.13  2.34  0.00  103.06

0 R: 1.000 P: 0.371 carry pockets ! room strap love back nice lot work
   1 R: 0.248 P: 0.111 pocket mouse power ipad charger netbook small room tablet cord
     11 R: 0.043 P: 0.043 ; & pro sleeve air perfectly protection retina price laptops
     12 R: 0.094 P: 0.094 sleeve protection inside air pro protect inch snug nice bit
   3 R: 0.184 P: 0.061 ! ... price $ bought amazon quality buy made reviews
     33 R: 0.068 P: 0.068 ! color love picture perfectly pink recommend perfect blue colors
     32 R: 0.054 P: 0.054 item smell arrived received ordered shipping days 'm return time
   5 R: 0.059 P: 0.033 zipper handle months broke strap year started years bought zippers
     51 R: 0.027 P: 0.027 back return time service money 'm years problem model customer
   4 R: 0.137 P: 0.046 bottom top piece corners plastic months part corner side speck
     41 R: 0.092 P: 0.092 cover keyboard mac color hard pro apple easy screen nice
{0: [1, 3, 4, 2], 1: [12, 13], 3: [33, 32, 31], 4: [41], 2: [21